In [ ]:
from keras.utils import image_dataset_from_directory
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [24]:
base_directory = './data/aiim-emotion-classification/'

In [25]:
# Documentation: https://keras.io/api/data_loading/image/
train_ds, validation_ds = image_dataset_from_directory(
    directory=base_directory + 'train/',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    batch_size=64,
    shuffle=True,
    seed=420,
    subset="both",
    validation_split=0.1,
    image_size=(100, 100))


class_names = train_ds.class_names

print(train_ds)

Found 9108 files belonging to 5 classes.
Using 8198 files for training.
Using 910 files for validation.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 100, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 5), dtype=tf.float32, name=None))>


# Convolutional Operation

## Convolutional Layer 1
* Number of input Channels: 1 (Greyscale)
* Number of output Channels: 32
* Kernel size: 3x3
* Stride: 1
* Padding: 1

## Max Pooling Layer 1
* Kernel size: 2x2
* Stride: 2
* Padding: 0

## Convolutional Layer 2
* Number of input Channels: 32
* Number of output Channels: 64
* kernel size: 3 (3x3 kernel)
* Stride: 1
* Padding 1

## Max Pooling Layer 2
* Kernel size: 2x2
* Stride: 2
* Padding: 0

## Flatten Layer
Converts the multi-dimensional output of the Convolutional and Pooling layers into a 1D vector.

## Fully Connected Layer 1
* Input features: 64 * 25 * 25 (output size after the second pooling layer, flattened)
* Output features: 128

## Fully Connected Layer 2
* Input features: 128
* Output features: Number of classes (5 different emotions)

In [26]:
model = Sequential()

# Convolutional Layer 1
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='Same', activation='relu', input_shape=(100,100,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

# Convolutional Layer 2
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))


C:\Users\Jasper\anaconda3\envs\AIIM\Lib\site-packages\keras\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Try Learning rate of 0.001

In [ ]:
# Define the optimizer
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)


# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)


# Training des Modells
history = model.fit(train_ds, validation_data=validation_ds, epochs=10, batch_size=64, callbacks=[early_stopping, reduce_lr])


Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 23s 162ms/step - accuracy: 0.2361 - loss: 5.2762 - val_accuracy: 0.2143 - val_loss: 2.7012 - learning_rate: 0.0010
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 18s 136ms/step - accuracy: 0.2198 - loss: 2.3838 - val_accuracy: 0.2132 - val_loss: 1.8330 - learning_rate: 0.0010
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 18s 141ms/step - accuracy: 0.2115 - loss: 1.8116 - val_accuracy: 0.2176 - val_loss: 1.6700 - learning_rate: 0.0010
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 17s 134ms/step - accuracy: 0.2202 - loss: 1.6900 - val_accuracy: 0.2154 - val_loss: 1.6701 - learning_rate: 0.0010
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 20s 158ms/step - accuracy: 0.2304 - loss: 1.6938 - val_accuracy: 0.2154 - val_loss: 1.7077 - learning_rate: 0.0010
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 146ms/step - accuracy: 0.2322 - loss: 1.7837 - val_accuracy: 0.2187 - val_loss: 2.0147 - learning_rate: 0.0010
Epoch 7/10
 67/129 ━━━━━━━━━━━━━━━━━━━━ 7s 129ms/step - accuracy: 0.20

In [ ]:
# Plot the loss and accuracy curves for training and validation 
plt.plot(history.history['val_loss'], color='b', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Create Submission

In [ ]:
test_ds = image_dataset_from_directory(
    directory=base_directory + 'test/',
    labels=None,
    label_mode='categorical',
    color_mode='grayscale',
    batch_size=32,
    shuffle=True,
    seed=69,
    image_size=(100, 100))


y_test = model.predict(test_ds)

# Bilddateinamen extrahieren
file_paths = test_ds.file_paths
file_names = [file_path.split('/')[-1] for file_path in file_paths]

y_pred_indices = np.argmax(y_test, axis=1)

# Numerische Labels in Kategorienamen umwandeln
y_pred_labels = [class_names[idx] for idx in y_pred_indices]

# DataFrame erstellen
df = pd.DataFrame({
    'Id': file_names,
    'emotions': y_pred_labels
})

# DataFrame als CSV speichern
df.to_csv('/submission.csv', index=False)

print(df.describe())